In [1]:
import pandas as pd
import datetime
import pandas_datareader.data as web
import numpy as np
import main,utils,sharpe,recommendations

In [2]:
stocks = utils.get_sp500()

Getting list of S&P stock symbols...


In [3]:
symb_list = utils.check_dtype_securities(stocks)
prices = utils.close_prices_loop('1d',symb_list)['Close'].min().reset_index()
prices.columns = ['Symbol','Close']

Getting closing prices...
Getting 0 to 4
0     MMM
1     ABT
2    ABBV
3    ABMD
4     ACN
Name: Symbol, dtype: object
Getting 5 to 9
5    ATVI
6    ADBE
7     AMD
8     AAP
9     AES
Name: Symbol, dtype: object
Getting 10 to 14
10     AFL
11       A
12     APD
13    AKAM
14     ALK
Name: Symbol, dtype: object
Getting 15 to 19
15     ALB
16     ARE
17    ALXN
18    ALGN
19    ALLE
Name: Symbol, dtype: object
Getting 20 to 24
20      LNT
21      ALL
22    GOOGL
23       MO
24     AMZN
Name: Symbol, dtype: object
Getting 25 to 29
25    AMCR
26     AEE
27     AAL
28     AEP
Name: Symbol, dtype: object


In [4]:
for index,row in prices.iterrows():
    if(row['Close']>100):
        stocks = stocks[stocks['Symbol']!=row['Symbol']]

In [5]:
stocks.reset_index(drop=True,inplace=True)

In [6]:
symb_list = utils.check_dtype_securities(stocks)

In [7]:
symb_list

0     ATVI
1      AMD
2      AES
3      AFL
4      ALK
5      LNT
6       MO
7     AMCR
8      AEE
9      AAL
10     AEP
Name: Symbol, dtype: object

In [10]:
def get_start(timeframe):
    if(timeframe=='1d'):
        start = (datetime.datetime.today()-datetime.timedelta(days=2)).strftime('%Y-%m-%d')
    elif(timeframe=='1w'):
        start = (datetime.datetime.today()-datetime.timedelta(days=8)).strftime('%Y-%m-%d')
    elif(timeframe=='1m'):
        start = (datetime.datetime.today()-datetime.timedelta(days=31)).strftime('%Y-%m-%d')
    elif(timeframe=='3m'):
        start = (datetime.datetime.today()-datetime.timedelta(days=93)).strftime('%Y-%m-%d')
    else:
        start = datetime.datetime.today().strftime('%Y-%m-%d')
    return start

def close_prices_loop(timeframe,security):
# minimizes the number of simultaneous price requests 
    print("Getting closing prices...")
    df = pd.DataFrame()
    num = len(security)
    residue = num%5
    incr = int((num-residue)/5)
    cnt = 0
    while(cnt<num-residue):
        print(f"Getting {cnt} to {cnt+incr-1}")
        df = pd.concat([get_close_prices(timeframe,security[cnt:cnt+incr]),df])
        cnt+=incr
    if(residue>0):
        print(f"Getting {cnt} to {cnt+residue}")
        df = pd.concat([get_close_prices(timeframe,security[cnt:cnt+residue]),df])
    return df

def get_close_prices(timeframe,security):    
    service = 'yahoo'
    start = get_start(timeframe)
    end = datetime.datetime.today().strftime('%Y-%m-%d')    
    try:
        print(security)
        df = np.round(web.DataReader(security,service,start,end)[['Close','Volume']],2)
        df.sort_values('Date',inplace = True)
    except ValueError as error:
        print("Couldn't connect to {} - {}".format(service,error))
    return df

In [12]:
df = close_prices_loop('1m',symb_list)

Getting closing prices...
Getting 0 to 1
0    ATVI
1     AMD
Name: Symbol, dtype: object
Getting 2 to 3
2    AES
3    AFL
Name: Symbol, dtype: object
Getting 4 to 5
4    ALK
5    LNT
Name: Symbol, dtype: object
Getting 6 to 7
6      MO
7    AMCR
Name: Symbol, dtype: object
Getting 8 to 9
8    AEE
9    AAL
Name: Symbol, dtype: object
Getting 10 to 11
10    AEP
Name: Symbol, dtype: object


In [15]:
df.fillna(0,inplace=True)

In [16]:
df.head()

Attributes Close                                                 ... Volume  \
Symbols      AAL  AEE    AEP  AES  AFL  ALK AMCR  AMD ATVI  LNT  ...    AEE   
Date                                                             ...          
2021-01-04   0.0  0.0  81.54  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
2021-01-05   0.0  0.0  81.43  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
2021-01-06   0.0  0.0  82.08  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
2021-01-07   0.0  0.0  79.89  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
2021-01-08   0.0  0.0  79.18  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   

Attributes                                                     
Symbols           AEP  AES  AFL  ALK AMCR  AMD ATVI  LNT   MO  
Date                                                           
2021-01-04  3232900.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-05  2212300.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-06  2713900.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-07  2764400.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-08  3059800.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 22 columns]